In [2]:
from sqlalchemy import create_engine, Column, String, DateTime, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
import datetime
from init_database import Transicion, HistorialPedido, Pedido
import os
from dotenv import load_dotenv

2025-03-17 18:27:42,233 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-17 18:27:42,235 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("pedidos")
2025-03-17 18:27:42,236 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-17 18:27:42,238 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("historial_pedidos")
2025-03-17 18:27:42,238 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-17 18:27:42,239 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("transiciones")
2025-03-17 18:27:42,241 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-17 18:27:42,242 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("proveedores")
2025-03-17 18:27:42,243 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-17 18:27:42,244 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ofertas")
2025-03-17 18:27:42,245 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-17 18:27:42,246 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("facturas")
2025-03-17 18:27:42,247 INFO sqlalchemy.engine

In [ ]:
from sqlalchemy import MetaData
from sqlalchemy_schemadisplay import create_schema_graph
from models.database import engine  # Asegúrate de importar el motor de tu BD

# Obtener metadatos de la base de datos
metadata = MetaData()
metadata.reflect(bind=engine)

# Crear el esquema de la base de datos
graph = create_schema_graph(
    metadata=metadata,
    show_datatypes=True,  # Muestra los tipos de datos
    show_indexes=True,  # Muestra los índices
    rankdir="LR",  # Orientación del gráfico (LR = Left to Right)
    concentrate=False  # No combina flechas en una sola línea
)

# Guardar como imagen
graph.write_png("database_schema.png")
print("✅ Esquema generado como 'database_schema.png'")


In [3]:
load_dotenv()
MODELO = os.getenv("MODELO")
KEY = os.getenv("KEY")
VERSION = os.getenv("VERSION")
ENDPOINT = os.getenv("ENDPOINT")

In [4]:
import openai
openai_client = openai.AzureOpenAI(
    api_key = KEY,
    api_version = VERSION,
    azure_endpoint = ENDPOINT
    )

In [5]:
messages=[{"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": "Como se dice perro en inglés?"},]

response = openai_client.chat.completions.create(
    model = MODELO,
    messages = messages,
    max_tokens=100
    )
response.choices[0].message.content

'"Perro" en inglés se dice "dog".'

In [35]:
 !pip install secure-smtplib


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def enviar_correo(destinatario, asunto, mensaje, remitente="blueprismautomail@gmail.com", clave="ocdd kthe cqeu qina"):
    """
    Envía un correo electrónico utilizando SMTP con Gmail.
    
    Parámetros:
    - destinatario: Email del destinatario.
    - asunto: Asunto del correo.
    - mensaje: Cuerpo del mensaje.
    - remitente: (Opcional) Tu dirección de correo de Gmail.
    - clave: (Opcional) Contraseña o App Password de tu cuenta de Gmail.
    """
    try:
        # Configuración del servidor SMTP de Gmail
        servidor_smtp = "smtp.gmail.com"
        puerto_smtp = 587

        # Crear el mensaje
        msg = MIMEMultipart()
        msg["From"] = remitente
        msg["To"] = destinatario
        msg["Subject"] = asunto
        msg.attach(MIMEText(mensaje, "plain"))

        # Conectar al servidor SMTP
        servidor = smtplib.SMTP(servidor_smtp, puerto_smtp)
        servidor.starttls()  # Seguridad con TLS
        servidor.login(remitente, clave)  # Autenticación

        # Enviar el correo
        servidor.sendmail(remitente, destinatario, msg.as_string())
        servidor.quit()

        print(f"✅ Correo enviado correctamente a {destinatario}")
    except Exception as e:
        print(f"❌ Error al enviar el correo: {e}")

# Ejemplo de uso
enviar_correo(
    destinatario="jorgevicentejuan@gmail.com",
    asunto="Prueba de correo",
    mensaje="Hola, esto es un correo de prueba enviado desde Python."
)


✅ Correo enviado correctamente a jorgevicentejuan@gmail.com


In [1]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END, Graph 
from models.database import *
from models.models import *
import os
from sqlalchemy.orm import Session
import json
from grafo.tools import *
from datetime import datetime
import time
#from dotenv import load_dotenv

In [11]:
def solicitud_compra(mensaje_usuario):
    prompt=f"""
    Eres un asistente que debe de clasificar el siguiente mensaje del usuario
    representa una solicitus de compra.

    Mensaje: {mensaje_usuario}.

    Responde únicamente con True si es una solicitud de compra o None si no lo es.
    No añadas ninguna otra palabra ni explicación
    """
    print("Se hace el invoke")
    respuesta = llm.invoke([HumanMessage(content = prompt)]).content.strip()
    print(respuesta)
    return respuesta


In [ ]:
print(solicitud_compra("Quiero comprar 500 portatiles del proveedor intel, tengo un presupuesto de 60000euros"))

Se hace el invoke
True
True


In [21]:
#INSERTAR DATOS DEL PROVEEDOR Y EL INVOKE DEL GRAFO

def invokar_agent(entrada):
    agente_proveedor = create_react_agent(llm ,tools=[])
    prompt = f''' 
    Eres el encargado de obtener la información de un proveedor.
    Se necesita conocer:
    - Nombre del proveedor 
    - Mail del proveedor
    - Contacto del proveedor
    Aqui tienes la información que tienes que obtener:
    {entrada}
    Devuelve unicamente un formato JSON como:
        "nombre": "...",
        "email": "...",
        "contacto": "..."
    '
    Una vez generado el json,simula que utilizas la herramienta insertar_proveedores para insertar el proveedor en la base de datos 
    y devuelve como resultado final el json que genera la tool.
    '''
    session = Session(bind=engine)
    response = agente_proveedor.invoke({'messages': [HumanMessage(content = prompt)]})
    contenido = response['messages'][-1].content
    print(f" el llm devuelve {contenido}")
    try: 
        print("ya se ha hehco el insert del nuevo proveedor, ahora falta meterle el proveedor para meterlo por el invoke si es nuevo o no")

        datos = json.loads(contenido)
        print(datos)
        nombre_proveedor = datos['nombre'].strip().lower()
        print(f"nombre provedor es {nombre_proveedor}")

        # 🔁 Volver a buscar el proveedor recién insertado
        proveedor = session.query(Proveedor).filter_by(nombre=nombre_proveedor.lower()).first()
        proveedor_id = proveedor.id_proveedor if proveedor else None
        print(f"✅ Proveedor '{proveedor_id}' encontrado en la base de datos.")
        session.close()
        # Resetear estado

        # Obtener mensaje original
        mensaje_lang = entrada
        print(mensaje_lang)
        ## INOKE DEL GRAFO
        pass
    except Exception as e:
        print(e)
                    

In [22]:
invokar_agent("Quiero comprar 500 portatiles del proveedor intel, tengo un presupuesto de 60000euros")

 el llm devuelve {
    "nombre": "Intel",
    "email": "info@intel.com",
    "contacto": "John Smith"
}
ya se ha hehco el insert del nuevo proveedor, ahora falta meterle el proveedor para meterlo por el invoke si es nuevo o no
{'nombre': 'Intel', 'email': 'info@intel.com', 'contacto': 'John Smith'}
nombre provedor es intel
✅ Proveedor '1' encontrado en la base de datos.
Quiero comprar 500 portatiles del proveedor intel, tengo un presupuesto de 60000euros


In [ ]:
## NODO 1

def obtener_datos_pedido(entrada_usuario):
    ''' Nodo que usa IA para obtener los datos del pedido del usuario, crea la cesta y el pedido con el estado inicial,
      y decide si el flujo valida el proveedor o graba la cesta.
    '''
    # Se crea la sesión de SQLAlchemy	
    session = Session(bind=engine)

    prompt = f''' 
    Eres el encargado de obtener la información de un pedido.
    Se necesita conocer:
    - Nombre del proveedor (si no se menciona, deja "desconocido")
    - Moneda en la que se realizó la compra utiliza el codigo de 3 letras
    - Presupuesto estimado
    - Producto a comprar
    - Si el proveedor es nuevo
    {entrada_usuario}

    Devuelve los datos en un formato claro como:
    "Proveedor: Proveedor X, Moneda: EUR, Presupuesto: 5000, Producto: [Producto Y], Cantidad: 10, Nuevo: True"
    '''
    response = llm.invoke(prompt)
    datos_ia = response.content.strip()

    print(f'IA: {datos_ia}')

    # Extraer los datos del texto
    datos_extraidos = {}
    for item in datos_ia.split(','):
        key, value = item.strip().split(':')
        datos_extraidos[key.lower().strip()] = value.strip()
    
    nombre_proveedor = datos_extraidos.get('proveedor', 'desconocido').strip().lower()
    codigo_moneda = datos_extraidos.get('moneda', 'EUR').strip().upper() 
    presupuesto = float(datos_extraidos.get('presupuesto', 1000)) # Por defecto 1000
    producto = datos_extraidos.get('producto', 'no especificado')
    cantidad = int(datos_extraidos.get('cantidad', 1)) # Por defecto 1
    es_nuevo_str = datos_extraidos.get('nuevo', 'false').strip().lower()
    es_nuevo = es_nuevo_str == "true"
    print(f"📦 Datos extraídos: Proveedor: {nombre_proveedor}, Moneda: {codigo_moneda}, Presupuesto: {presupuesto}")
    
    # Se implementa el estado inicial en funcion de si es nuevo o no el proveedor
    if es_nuevo:
        estado_inicial_nombre = "pendiente_condicionada"
    else:
        estado_inicial_nombre = "pendiente_no_condicionada"

    proveedor = session.query(Proveedor).filter_by(nombre=nombre_proveedor.lower()).first()
    proveedor_id = proveedor.id_proveedor if proveedor else None
    print(f"✅ Proveedor '{proveedor_id}' encontrado en la base de datos.")

    # Obtener estado inicial correspondiente
    estado_inicial = session.query(Estado).filter_by(nombre=estado_inicial_nombre).first()
    if not estado_inicial:
        print(f"⚠️ Estado '{estado_inicial_nombre}' no encontrado en la base de datos.")
        return {"error": "Estado inicial no encontrado"}

    # Verificar que la moneda existe en la BD
    moneda = session.query(Moneda).filter(Moneda.codigo == codigo_moneda).first()
    moneda_id = moneda.id_moneda if moneda else None

    if not moneda:
        print(f"⚠️ Moneda '{codigo_moneda}' no encontrada. Se usará EUR por defecto.")
        moneda = session.query(Moneda).filter_by(codigo="EUR").first()
        moneda_id = moneda.id_moneda

    # Crear la cesta y el pedido
    nueva_cesta = Cesta(
        nombre="Cesta generada automáticamente",
        descripcion="Pedido generado automáticamente",
        tipo_compra="ordinaria",
        presupuesto=presupuesto,
        usuario_sap_id=1,
        proveedor_id=proveedor_id,
        moneda_id=moneda_id,
        contrato_id=None
    )
    session.add(nueva_cesta)
    session.commit()

    print(f"✅ Cesta creada con ID: {nueva_cesta.id_cesta}")

    nuevo_pedido = Pedido(
        posicion=1,
        tipo="Ordinario",
        pedido_tipoimp="K",
        descripcion=f"Compra de {datos_extraidos.get('producto', 'productos no especificados')}",
        id_proveedor=proveedor_id,
        cesta_id=nueva_cesta.id_cesta, 
        estado_tramitacion_id=estado_inicial.id_estado,
        creador_id=1,
        moneda_id=moneda_id
    )
    session.add(nuevo_pedido)
    session.commit()

    # Se extraen las variables antes de cerrar la sesion 

    pedido_id = nuevo_pedido.id_pedido
    cesta_id = nueva_cesta.id_cesta
    session.close()

    return {

        "id_pedido": pedido_id,
        "id_proveedor": proveedor_id, # Ya no intenta acceder a un objeto desvinculado
        "nombre_proveedor": nombre_proveedor,
        "id_cesta": cesta_id,
        "id_moneda": moneda_id,  # Ya no intenta acceder a un objeto desvinculado
        "estado_actual": estado_inicial_nombre,
        "id_usuario": 1,
        "nombre_cesta": "Cesta generada automáticamente",
        "tipo_compra": "ordinaria",
        "presupuesto": presupuesto,
        "descripcion": producto,
        "pedido_tipoimp": "K",
        "cantidad": cantidad,
        "es_nuevo": es_nuevo  # Indica si el proveedor debe validarse antes de continuar
    }